In [1]:
import json
import glob
import tqdm
import os
import time
from langchain.prompts import PromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI
from tenacity import (
    retry,
    stop_after_attempt,
    wait_random_exponential,
    retry_if_exception_type
)
from langchain_core.prompts import ChatPromptTemplate

from framework import get_language_name

d:\ws\gh\eamt\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
d:\ws\gh\eamt\.venv\Lib\site-packages\torchmetrics\utilities\imports.py:23: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import DistributionNotFound, get_distribution


In [ ]:
os.environ['GOOGLE_API_KEY'] = 'YOUR_API_KEY'

In [4]:
model_name = "gemini-2.0-flash"

llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
)

### Rate Limit Configuration for Gemini 2.0 Flash (Free Tier)
Based on Google AI for Developers documentation for gemini-2.0-flash free tier:
RPM (Requests Per Minute): 15
RPD (Requests Per Day): 1,500
To stay within 15 RPM, we need to ensure at least 4 seconds delay between requests.

In [5]:
REQUESTS_PER_MINUTE_LIMIT = 15
DELAY_BETWEEN_REQUESTS_SECONDS = 120 / REQUESTS_PER_MINUTE_LIMIT

BATCH_SIZE = 50

In [6]:
# Counter for API requests
api_request_count = 0

In [7]:
input_data_folder = "./data/semeval.validation.v2-889a1492ba6c3791baa8f4224bc8e685/validation"
jsonl_files = glob.glob(f"{input_data_folder}/*.jsonl")

output_prediction_dir = os.path.join("data/predictions", model_name, "validation")
os.makedirs(output_prediction_dir, exist_ok=True)

jsonl_files = glob.glob(f"{input_data_folder}/*.jsonl")
jsonl_files

['./data/semeval.validation.v2-889a1492ba6c3791baa8f4224bc8e685/validation\\ar_AE.jsonl',
 './data/semeval.validation.v2-889a1492ba6c3791baa8f4224bc8e685/validation\\de_DE.jsonl',
 './data/semeval.validation.v2-889a1492ba6c3791baa8f4224bc8e685/validation\\es_ES.jsonl',
 './data/semeval.validation.v2-889a1492ba6c3791baa8f4224bc8e685/validation\\fr_FR.jsonl',
 './data/semeval.validation.v2-889a1492ba6c3791baa8f4224bc8e685/validation\\it_IT.jsonl']

In [8]:
from framework import download_comet_model
comet_model = download_comet_model()

Fetching 5 files: 100%|██████████| 5/5 [00:00<00:00, 9412.71it/s]
Lightning automatically upgraded your loaded checkpoint from v1.8.3.post1 to v2.5.1.post0. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint C:\Users\ic019797\.cache\huggingface\hub\models--Unbabel--wmt22-comet-da\snapshots\2760a223ac957f30acfb18c8aa649b01cf1d75f2\checkpoints\model.ckpt`
Encoder model frozen.
d:\ws\gh\eamt\.venv\Lib\site-packages\pytorch_lightning\core\saving.py:195: Found keys that are not in the model state dict but in the checkpoint: ['encoder.model.embeddings.position_ids']


In [9]:
from framework import calculate_comet_scores, calculate_meta_score

def calculate_scores(template_id):
    scores_dir = os.path.join(output_prediction_dir, template_id, "scores")

    if not os.path.exists(scores_dir):
        os.makedirs(scores_dir, exist_ok=True)

    for file_path in jsonl_files:
        references_path = file_path
        filename = os.path.basename(file_path)
        predictions_path = os.path.join(output_prediction_dir, template_id, filename)

        comet_score = calculate_comet_scores(
            comet_model, 
            references_path, 
            predictions_path
        )

        correct_instances, total_instances, meta_score = calculate_meta_score(
            references_path,
            predictions_path)

        evaluation_results = {
            "correct_instances": correct_instances,
            "total_instances": total_instances,
            "comet_score": comet_score,
            "meta_score": meta_score
        }

        evaluation_output_path = os.path.join(scores_dir, f"{os.path.splitext(filename)[0]}.json")
        with open(evaluation_output_path, 'w', encoding='utf-8') as json_file:
            json.dump(evaluation_results, json_file, ensure_ascii=False, indent=4)

# Zero shot prompting

### Prompt Template for Batched Translation

This defines how multiple input texts and target language are formatted for the LLM.
We use a unique delimiter to separate sentences in the input and expect the same in output.

In [15]:
ZERO_SHOT_PROMPT_TEMPLATE_STRING_1 = """"
Your task is to translate the following English sentences into {target_language}.
Input sentences are provided below as a JSON array of objects, each with an "id" and a "text" field.
Provide the translations as a JSON array of objects, where each object contains the original "id" and its "translation".
Maintain the original order of sentences from the input. Please return the json array without any additional text or formatting.

Input JSON:
{source_texts_json}
"""

ZERO_SHOT_PROMPT_TEMPLATE_STRING_2 = """"
Your task is to translate the following English sentences into {target_language}.
Input sentences are provided below as a JSON array of objects, each with an "id" and a "text" field.
Ensure that **all named entities** (e.g., people, organizations, locations, product names) are translated **correctly and consistently** into the target language.
Provide the translations as a JSON array of objects, where each object contains the original "id" and its "translation".
Maintain the original order of sentences from the input. Please return the json array without any additional text or formatting.

Input JSON:
{source_texts_json}
"""


In [ ]:
# Translation function with exponential backoff retry logic
@retry(
    # Wait for a random exponential amount between 4 seconds (min) and 60 seconds (max).
    # This prevents hammering the API and adds jitter to avoid synchronized retries.
    wait=wait_random_exponential(multiplier=1, min=DELAY_BETWEEN_REQUESTS_SECONDS, max=60),
    # Stop trying after 5 attempts to prevent infinite loops on persistent errors.
    stop=stop_after_attempt(5),
    # Retry if any Exception occurs. You could make this more specific
    # if the API returns a particular error type for rate limits (e.g., HTTP 429).
    retry=retry_if_exception_type(Exception)
)
def get_translated_content_with_retries(source_records_batch, target_language, llm_instance, template):
    """
    Translates a batch of source_texts using the provided LLM instance.
    Includes robust retry logic with exponential backoff for API stability.
    """
    global api_request_count

    input_json_for_prompt = [{"id": rec['id'], "text": rec['source']} for rec in source_records_batch]
    source_texts_json_str = json.dumps(input_json_for_prompt, ensure_ascii=False)

    prompt_to_send = template.format(
        source_texts_json=source_texts_json_str,
        target_language=target_language
    )
    
    response = llm_instance.invoke(prompt_to_send)
    api_request_count += 1

    try:
        cleaned_content = response.content.strip()
        if cleaned_content.startswith("```json") and cleaned_content.endswith("```"):
            cleaned_content = cleaned_content[7:-3].strip()
        
        translated_outputs = json.loads(cleaned_content)
        
        if not isinstance(translated_outputs, list) or \
           not all(isinstance(item, dict) and 'id' in item and 'translation' in item for item in translated_outputs):
            raise ValueError("Model did not return a valid JSON array of translation objects.")

        translated_dict = {item['id']: item['translation'] for item in translated_outputs}

        ordered_translations = []
        for record in source_records_batch:
            ordered_translations.append(translated_dict.get(record['id'], "ERROR: ID not found in JSON output"))
        
        return ordered_translations

    except json.JSONDecodeError as e:
        print(f"JSON Decode Error: {e}. Raw response: {response.content.strip()}")
        # Fallback to a list of error messages for the batch
        return ["ERROR: Invalid JSON response from model"] * len(source_records_batch)
    except ValueError as e:
        print(f"Value Error (JSON format issue): {e}. Raw response: {response.content.strip()}")
        return ["ERROR: Invalid JSON structure from model"] * len(source_records_batch)
    except Exception as e:
        print(f"An unexpected error occurred while processing model output: {e}. Raw response: {response.content.strip()}")
        return ["ERROR: Unexpected issue processing model output"] * len(source_records_batch)


In [10]:
def zero_shot_eval(template, template_id):
    overall_pbar = tqdm.tqdm(jsonl_files, desc="Processing files")

    output_prediction_dir_1 = os.path.join(output_prediction_dir, template_id)
    os.makedirs(output_prediction_dir_1, exist_ok=True)

    for file_path in overall_pbar:
        filename = os.path.basename(file_path)
        outfile_path = os.path.join(output_prediction_dir_1, filename)

        data_to_translate = []
        with open(file_path, 'r', encoding='utf-8') as f:
            for line in f:
                data_to_translate.append(json.loads(line))

        translated_results_for_file = []

        for i in tqdm.tqdm(range(0, len(data_to_translate), BATCH_SIZE), desc=f"Translating {filename} in batches", leave=False):
            batch_records = data_to_translate[i : i + BATCH_SIZE]

            target_locale = batch_records[0]['target_locale']
            target_language = get_language_name(target_locale)

            try:
                translated_texts_batch = get_translated_content_with_retries(
                    batch_records, target_language, llm, template
                )

                for j, record in enumerate(batch_records):
                    record_id = record['id']
                    source_text = record['source']
                    source_locale = record['source_locale']
                    translated_text = translated_texts_batch[j] if j < len(translated_texts_batch) else "ERROR: Translation missing"
                    translated_results_for_file.append({
                        "id": record_id,
                        "source_language": get_language_name(source_locale),
                        "target_language": target_language,
                        "text": source_text,
                        "prediction": translated_text,
                    })

            except Exception as e:
                print(f"\nCRITICAL ERROR: Failed to translate a batch starting with ID '{batch_records[0]['id']}' after multiple retries. Error: {e}")
                for record in batch_records:
                    translated_results_for_file.append({
                        "id": record['id'],
                        "source_language": get_language_name(record['source_locale']),
                        "target_language": get_language_name(record['target_locale']),
                        "text": record['source'],
                        "prediction": "ERROR: Batch translation failed due to API issues/rate limits.",
                    })

            if i + BATCH_SIZE < len(data_to_translate) or overall_pbar.n < len(jsonl_files):
                 time.sleep(DELAY_BETWEEN_REQUESTS_SECONDS)

        with open(outfile_path, 'w', encoding='utf-8') as f:
            for res in translated_results_for_file:
                f.write(json.dumps(res, ensure_ascii=False) + '\n')

        print(f"\nTranslations for {filename} saved to {outfile_path}")

    overall_pbar.close()
    print("All files processed and translations saved.")

In [ ]:
zero_shot_prompt_template = PromptTemplate(
    input_variables=["source_texts_json", "target_language"],
    template=ZERO_SHOT_PROMPT_TEMPLATE_STRING_1,
)

zero_shot_eval(zero_shot_prompt_template, "zero-shot-1")
calculate_scores("zero-shot-1")

In [18]:
zero_shot_prompt_template = PromptTemplate(
    input_variables=["source_texts_json", "target_language"],
    template=ZERO_SHOT_PROMPT_TEMPLATE_STRING_2,
)

zero_shot_eval(zero_shot_prompt_template, "zero-shot-2")
calculate_scores("zero-shot-2")

[
  {"id": "Q101438737_0", "translation": "في أي نوع يندرج كتاب أرض الميعاد؟"},
  {"id": "Q101438737_1", "translation": "من هو مؤلف العمل الفني أرض الميعاد؟"},
  {"id": "Q101438737_2", "translation": "متى تم نشر كتاب أرض الميعاد؟"},
  {"id": "Q10298666_0", "translation": "ما هو نوع فيلم كيف تروض تنينك 2؟"},
  {"id": "Q10298666_1", "translation": "في أي عام تم إصدار فيلم كيف تروض تنينك 2؟"},
  {"id": "Q10298666_2", "translation": "هل فيلم كيف تروض تنينك 2 فيلم رسوم متحركة؟"},
  {"id": "Q1031002_0", "translation": "أي فنان قصص مصورة بلجيكي شهير ابتكر مغامرات تان تان؟"},
  {"id": "Q1031002_1", "translation": "ما هو نوع مغامرات تان تان؟"},
  {"id": "Q1031002_2", "translation": "كم عدد المواسم الموجودة في مغامرات تان تان؟"},
  {"id": "Q1044613_0", "translation": "ما هي الوظيفة الرئيسية لمحطة هلسنكي المركزية للسكك الحديدية؟"},
  {"id": "Q1044613_1", "translation": "كيف تصف الهندسة المعمارية لمحطة هلسنكي المركزية للسكك الحديدية؟"},
  {"id": "Q1044613_2", "translation": "كم عدد الأرصفة الموجود

[
  {"id": "Q1137661_0", "translation": "هل يمكنك تقديم وصف موجز لدير غراتشانيكا؟"},
  {"id": "Q1137661_1", "translation": "ما هو نوع المكان الذي يمثله دير غراتشانيكا؟"},
  {"id": "Q1137661_2", "translation": "ما هي أهمية دير غراتشانيكا؟"},
  {"id": "Q11446789_0", "translation": "هل تدور أحداث Elegant Yokai Apartment Life في إطار حديث أم تاريخي؟"},
  {"id": "Q11446789_1", "translation": "من هو مؤلف Elegant Yokai Apartment Life؟"},
  {"id": "Q11446789_2", "translation": "كم عدد الكتب الموجودة في سلسلة Elegant Yokai Apartment Life؟"},
  {"id": "Q1152214_0", "translation": "ما هي أهمية عنوان The Whisperer in Darkness بالنسبة للرواية القصيرة؟"},
  {"id": "Q1152214_1", "translation": "من هو مؤلف الرواية القصيرة The Whisperer in Darkness؟"},
  {"id": "Q1152214_2", "translation": "كيف يمكنك تلخيص حبكة الرواية القصيرة The Whisperer in Darkness؟"},
  {"id": "Q1167737_0", "translation": "ما هو التركيز الرئيسي لكتاب مستقبل وهم لـ سيغموند فرويد؟"},
  {"id": "Q1167737_1", "translation": "كيف تصف مو

[
  {"id": "Q1248011_2", "translation": "كم عدد مواسم المسلسل التلفزيوني أبنائي الثلاثة التي تم إنتاجها؟"},
  {"id": "Q1254986_0", "translation": "من هم الشخصيات الرئيسية في دراغون بول زد: شجرة القوة؟"},
  {"id": "Q1254986_1", "translation": "متى تم إصدار دراغون بول زد: شجرة القوة؟"},
  {"id": "Q1254986_2", "translation": "ما هو نوع دراغون بول زد: شجرة القوة؟"},
  {"id": "Q13099103_0", "translation": "كيف تقارن طعم حليب القنب بأنواع الحليب النباتية الأخرى؟"},
  {"id": "Q13099103_1", "translation": "ما هي المكونات الرئيسية في حليب القنب؟"},
  {"id": "Q13099103_2", "translation": "هل حليب القنب مناسب للأشخاص الذين لديهم قيود غذائية؟"},
  {"id": "Q131074_0", "translation": "متى تم إصدار فيلم سيد الخواتم: عودة الملك؟"},
  {"id": "Q131074_1", "translation": "من أخرج فيلم 2003، سيد الخواتم: عودة الملك؟"},
  {"id": "Q131074_2", "translation": "إلى أي نوع ينتمي فيلم سيد الخواتم: عودة الملك؟"},
  {"id": "Q1316142_0", "translation": "هل يمكنك تقديم وصف موجز لشخصية ألفي أتكنز؟"},
  {"id": "Q13161

[
  {"id": "Q1545219_0", "translation": "هل الأب براون شخصية حقيقية أم خيالية؟"},
  {"id": "Q1545219_1", "translation": "ما هي مهنة الأب براون؟"},
  {"id": "Q1545219_2", "translation": "في أي بلد تم إنشاء الأب براون؟"},
  {"id": "Q1548310_0", "translation": "هل يمكن للجمهور زيارة مسجد بروكسل الكبير؟"},
  {"id": "Q1548310_1", "translation": "أين يقع مسجد بروكسل الكبير؟"},
  {"id": "Q1548310_2", "translation": "ما هو الغرض من مسجد بروكسل الكبير؟"},
  {"id": "Q15547365_0", "translation": "كيف يعكس مسجد ناجياينغ العمارة الصينية؟"},
  {"id": "Q15547365_1", "translation": "ما هو الغرض الأساسي من مسجد ناجياينغ؟"},
  {"id": "Q15547365_2", "translation": "متى تم بناء مسجد ناجياينغ؟"},
  {"id": "Q15616479_0", "translation": "من هم الشخصيات الرئيسية في بطاريق مدغشقر؟"},
  {"id": "Q15616479_1", "translation": "متى تم إصدار فيلم بطاريق مدغشقر؟"},
  {"id": "Q15616479_2", "translation": "ما هو نوع فيلم بطاريق مدغشقر؟"},
  {"id": "Q1572056_0", "translation": "ما هي الرياضات التي يتم تمثيلها عادةً في ت

[
  {
    "id": "Q1759977_2",
    "translation": "متى نُشرت رواية Revelation Space لأول مرة؟"
  },
  {
    "id": "Q176061_0",
    "translation": "بأية طرق تساهم لغة تصميم الواقع الافتراضي (Virtual Reality Modeling Language) في تطوير تطبيقات الواقع الافتراضي؟"
  },
  {
    "id": "Q176061_1",
    "translation": "كيف تعزز لغة تصميم الواقع الافتراضي (Virtual Reality Modeling Language) تجربة المستخدم في البيئات الافتراضية التفاعلية؟"
  },
  {
    "id": "Q176061_2",
    "translation": "ما هي الميزات الرئيسية للغة تصميم الواقع الافتراضي (Virtual Reality Modeling Language) التي تجعلها تنسيق ملفات شائعًا للرسومات ثلاثية الأبعاد؟"
  },
  {
    "id": "Q179468_0",
    "translation": "من أين كان شلمنصر الرابع (Shalmaneser IV)؟"
  },
  {
    "id": "Q179468_1",
    "translation": "بماذا اشتهر شلمنصر الرابع (Shalmaneser IV)؟"
  },
  {
    "id": "Q179468_2",
    "translation": "متى حكم شلمنصر الرابع (Shalmaneser IV)؟"
  },
  {
    "id": "Q1810196_0",
    "translation": "ما هو نوع رواية Dandelion Wine؟"

...

Average COMET score: 92.33
Loaded 739 instances.
Loaded 739 predictions.
All references have a corresponding prediction
Created 1316 instances


Predicting DataLoader 0: 100%|██████████| 42/42 [03:22<00:00,  4.83s/it]
Using default `ModelCheckpoint`. Consider installing `litmodels` package to enable `LitModelCheckpoint` for automatic upload to the Lightning model registry.
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


Average COMET score: 88.20
Loaded 724 instances.
Loaded 724 predictions.
All references have a corresponding prediction
Created 1268 instances


Predicting DataLoader 0: 100%|██████████| 40/40 [02:51<00:00,  4.28s/it]

Average COMET score: 91.51
Loaded 730 instances.
Loaded 730 predictions.


In [19]:
api_request_count

171

# Few Shot prompting

In [33]:
FEW_SHOT_PROMPT_TEMPLATE_STRING = """
Your task is to translate English sentences into {target_language}.
Below are a few examples of English sentences with named entities translated correctly into {target_language}.
Pay close attention to how named entities (e.g., people, organizations, locations, product names) are translated correctly and consistently.

Examples:
{few_shot_examples_json}

Now, translate the following new English sentences into {target_language}.
Provide the translations as a JSON array of objects, where each object contains the original "id" and its "translation" (ensure the key name is "translation").
Maintain the original order of sentences from the input. Please return the json array without any additional text or formatting.

Input JSON:
{source_texts_json}
"""

In [34]:
few_shot_examples_dict = {
    'ar': [
        {'source': 'What is the seventh tallest mountain in North America?', 'target': 'ما سابع أعلى جبل في أمريكا الشمالية؟'},
        {'source': 'Which actor was the star of Titanic and was born in Los Angeles, California?', 'target': 'مَنْ الممثل الذي لعب دور البطولة في فيلم "تيتانيك" وهو من مواليد لوس أنجلوس بكاليفورنيا؟'},
        {'source': 'What year was the first book of the A Song of Ice and Fire series published?', 'target': 'في أي عام تم نشر أول كتاب من سلسلة "أغنية الجليد والنار"؟'},
        {'source': 'Who is the youngest current US governor?', 'target': 'مَن أصغر حاكم ولاية أمريكي حالٍ؟'},
        {'source': 'How long did it take to build the Lincoln Memorial?', 'target': 'كم من الوقت استغرق بناء نصب لينكولن التذكاري؟'},
        {'source': 'Has Bernie Sanders ever been president of the United States?', 'target': 'هل كان بيرني ساندرز رئيسًا للولايات المتحدة يومًا ما؟'},
        {'source': 'Which actor was Stephenie Meyers first choice to play Edward Cullen in the movie Twilight?', 'target': 'مَن الممثل الذي وقع عليه الاختيار الأول للكاتبة ستيفاني ماير للعب دور "إدوارد كولن" في فيلم "الشفق"؟'},
        {'source': 'Which river is longer than the Mississippi River?', 'target': 'أي نهر أطول من نهر المسيسيبي؟'},
        {'source': 'What is the latest US state to be admitted to the union that is not Hawaii?', 'target': 'ما آخر ولاية أمريكية تم قبولها في الاتحاد، بخلاف هاواي؟'},
        {'source': 'What is the longest lake in the world?', 'target': 'ما أطول بحيرة في العالم؟'},
    ],
    'de': [
        {'source': 'What is the seventh tallest mountain in North America?', 'target': 'Wie heißt der siebthöchste Berg Nordamerikas?'},
        {'source': 'What year was the first book of the A Song of Ice and Fire series published?', 'target': 'In welchem Jahr wurde das erste Buch der Reihe "Das Lied von Eis und Feuer" veröffentlicht?'},
        {'source': 'Who is the youngest current US governor?', 'target': 'Wer ist derzeit der jüngste amerikanische Gouverneur?'},
        {'source': 'Has Bernie Sanders ever been president of the United States?', 'target': 'War Bernie Sanders jemals Präsident der Vereinigten Staaten?'},
        {'source': 'Which actor was Stephenie Meyers first choice to play Edward Cullen in the movie Twilight?', 'target': 'Welcher Schauspieler war Stephanie Meyers erste Wahl für die Rolle des Edward Cullen in dem Film Twilight – Biss zum Morgengrauen?'},
        {'source': 'Which river is longer than the Mississippi River?', 'target': 'Welcher Fluss ist länger als der Mississippi'},
        {'source': 'What is the longest lake in the world?', 'target': 'Welcher See ist der längste der Welt?'},
        {'source': 'Is Texas the largest state in US?', 'target': 'Ist Texas der größte Bundesstaat in den Vereinigten Staaten?'},
        {'source': 'Who was the president of Argentina from 1989 to 1999?', 'target': 'Wer war von 1989 - 1999 Präsident von Argentinien?'},
        {'source': 'Who was a member of the 2004 U.S. Olympic mens swim team and born in Baltimore, Maryland?', 'target': 'Wer gehörte 2004 zur olympischen, Schwimmstaffel der Vereinigten Staaten und wurde in Baltimore, Maryland geboren?'},
    ],
    'es': [
        {'source': 'Which actor was the star of Titanic and was born in Los Angeles, California?', 'target': '¿Qué actor protagonizó Titanic y nació en Los Ángeles, California?'},
        {'source': 'What year was the first book of the A Song of Ice and Fire series published?', 'target': '¿En qué año se publicó el primer libro de la saga Canción de hielo y fuego?'},
        {'source': 'Which US president has had the most votes?', 'target': '¿Qué presidente de Estados Unidos obtuvo más votos?'},
        {'source': 'How long did it take to build the Lincoln Memorial?', 'target': '¿En cuánto tiempo se construyó el Monumento a Lincoln?'},
        {'source': 'Has Bernie Sanders ever been president of the United States?', 'target': '¿Bernie Sanders ha sido alguna vez presidente de los Estados Unidos?'},
        {'source': 'Which actor was Stephenie Meyers first choice to play Edward Cullen in the movie Twilight?', 'target': '¿Qué actor fue la primera opción de Stephanie Meyer para interpretar a Edward Cullen en la película Crepúsculo?'},
        {'source': 'What is the latest US state to be admitted to the union that is not Hawaii?', 'target': '¿Cuál es el último estado de Estados Unidos en ser incorporado a la unión aparte de Hawái?'},
        {'source': 'What is the longest lake in the world?', 'target': '¿Cuál es el lago más largo del mundo?'},
        {'source': 'Is Texas the largest state in US?', 'target': '¿Tejas es el estado más grande de Estados Unidos?'},
        {'source': 'How many times have the Los Angeles Dodgers lost the World Series?', 'target': '¿Cuántas veces perdieron los Dodgers de Los Ángeles la Serie Mundial?'},
    ],
    'fr': [
        {'source': 'What is the seventh tallest mountain in North America?', 'target': 'Quelle est la septième plus haute montagne d’Amérique du Nord ?'},
        {'source': 'Who is the youngest current US governor?', 'target': 'Qui est l’actuel plus jeune gouverneur américain ?'},
        {'source': 'Has Bernie Sanders ever been president of the United States?', 'target': 'Bernie Sanders a-t-il déjà été Président des États-Unis ?'},
        {'source': 'Which actor was Stephenie Meyers first choice to play Edward Cullen in the movie Twilight?', 'target': 'Quel acteur Stephanie Meyer a-t-elle choisi en premier pour jouer le rôle d’Edward Cullen dans le film Twilight ?'},
        {'source': 'Which river is longer than the Mississippi River?', 'target': 'Quel fleuve est plus long que le Mississippi ?'},
        {'source': 'What is the latest US state to be admitted to the union that is not Hawaii?', 'target': 'Quel est le dernier État Américain à avoir été admis dans l’Union et qui n’est pas Hawaï ?'},
        {'source': 'What is the longest lake in the world?', 'target': 'Quel est le lac le plus long du monde ?'},
        {'source': 'How many times have the Los Angeles Dodgers lost the World Series?', 'target': 'Combien de fois les Dodgers de Los Angeles ont-ils perdu dans la série mondiale ?'},
        {'source': 'Who was the president of Argentina from 1989 to 1999?', 'target': 'Qui était le Président de l’Argentine, de mille neuf cent quatre-vingt-neuf à mille neuf cent quatre-vingt-dix-neuf ?'},
        {'source': 'Who was a member of the 2004 U.S. Olympic mens swim team and born in Baltimore, Maryland?', 'target': 'Quel membre de l’équipe olympique masculine de natation des États-Unis de 2004 est né à Baltimore, dans le Maryland ?'},
    ],
    'it': [
        {'source': 'What year was the first book of the A Song of Ice and Fire series published?', 'target': 'In che anno venne pubblicato il primo libro della serie Cronache del Ghiaccio e del Fuoco?'},
        {'source': 'Which actor was Stephenie Meyers first choice to play Edward Cullen in the movie Twilight?', 'target': 'Quale attore era stata la prima scelta di Stephanie Meyer per interpretare Edward Cullen nel film Twilight?'},
        {'source': 'Which river is longer than the Mississippi River?', 'target': 'Quale fiume è più lungo del Mississippi?'},
        {'source': 'What is the longest lake in the world?', 'target': 'Qual è il lago più lungo del mondo?'},
        {'source': 'How many times have the Los Angeles Dodgers lost the World Series?', 'target': 'Quante volte i Los Angeles Dodgers hanno perso la Worls Series?'},
        {'source': 'Who was a member of the 2004 U.S. Olympic mens swim team and born in Baltimore, Maryland?', 'target': 'Chi era un membro della squadra olimpica di nuoto maschile americana del 2004 ed è nato a Baltimora, Maryland?'},
        {'source': 'Has there ever been a Christian U.S. senator?', 'target': 'Cè mai stato un senatore americano cristiano?'},
        {'source': 'How many Pulitzer Prizes has Colson Whitehead won?', 'target': 'Quanti premi Pulitzer ha vinto Colson Whitehead?'},
        {'source': 'Which movie was released first, Avengers: Endgame or Joker?', 'target': 'Quale film è uscito prima, Avengers Endgame o Joker?'},
        {'source': 'Was Friday the 13th released the same year as Halloween?', 'target': 'Venerdì 13 è uscito lo stesso anno di Halloween?'},
    ]
}

In [39]:
from tenacity import retry, stop_after_attempt, wait_random_exponential, retry_if_exception_type

@retry(
    wait=wait_random_exponential(multiplier=1, min=DELAY_BETWEEN_REQUESTS_SECONDS, max=60),
    stop=stop_after_attempt(5),
    retry=retry_if_exception_type(Exception)
)
def get_translated_content_with_retries_few_shot(source_records_batch, target_language, llm_instance, template, few_shot_examples):
    """
    Translates a batch of source_texts using the provided LLM instance and few-shot examples.
    Includes robust retry logic with exponential backoff for API stability.
    """
    global api_request_count

    input_json_for_prompt = [{"id": rec['id'], "text": rec['source']} for rec in source_records_batch]
    source_texts_json_str = json.dumps(input_json_for_prompt, ensure_ascii=False)
    few_shot_examples_json = json.dumps(few_shot_examples, ensure_ascii=False)

    prompt_to_send = template.format(
        source_texts_json=source_texts_json_str,
        target_language=target_language,
        few_shot_examples_json=few_shot_examples_json
    )

    print(f"Sending prompt to model: {prompt_to_send}")  # Log the first 1000 characters of the prompt for debugging

    response = llm_instance.invoke(prompt_to_send)
    api_request_count += 1

    try:
        cleaned_content = response.content.strip()
        if cleaned_content.startswith("```json") and cleaned_content.endswith("```"):
            cleaned_content = cleaned_content[7:-3].strip()

        translated_outputs = json.loads(cleaned_content)

        print(f"Model response: {translated_outputs}")  # Log the model response for debugging

        if not isinstance(translated_outputs, list) or \
           not all(isinstance(item, dict) and 'id' in item and 'translation' in item for item in translated_outputs):
            raise ValueError("Model did not return a valid JSON array of translation objects.")

        translated_dict = {item['id']: item['translation'] for item in translated_outputs}

        ordered_translations = []
        for record in source_records_batch:
            ordered_translations.append(translated_dict.get(record['id'], "ERROR: ID not found in JSON output"))

        return ordered_translations

    except json.JSONDecodeError as e:
        print(f"JSON Decode Error: {e}. Raw response: {response.content.strip()}")
        return ["ERROR: Invalid JSON response from model"] * len(source_records_batch)
    except ValueError as e:
        print(f"Value Error (JSON format issue): {e}. Raw response: {response.content.strip()}")
        return ["ERROR: Invalid JSON structure from model"] * len(source_records_batch)
    except Exception as e:
        print(f"An unexpected error occurred while processing model output: {e}. Raw response: {response.content.strip()}")
        return ["ERROR: Unexpected issue processing model output"] * len(source_records_batch)

In [40]:
def few_shot_eval(template, template_id, few_shot_examples):
    overall_pbar = tqdm.tqdm(jsonl_files, desc="Processing files (few-shot)")

    output_prediction_dir_1 = os.path.join(output_prediction_dir, template_id)
    os.makedirs(output_prediction_dir_1, exist_ok=True)

    for file_path in overall_pbar:
        filename = os.path.basename(file_path)
        outfile_path = os.path.join(output_prediction_dir_1, filename)

        data_to_translate = []
        with open(file_path, 'r', encoding='utf-8') as f:
            for line in f:
                data_to_translate.append(json.loads(line))

        translated_results_for_file = []

        for i in tqdm.tqdm(range(0, len(data_to_translate), BATCH_SIZE), desc=f"Translating {filename} in batches (few-shot)", leave=False):
            batch_records = data_to_translate[i : i + BATCH_SIZE]

            target_locale = batch_records[0]['target_locale']
            target_language = get_language_name(target_locale)

            try:
                translated_texts_batch = get_translated_content_with_retries_few_shot(
                    batch_records, target_language, llm, template, few_shot_examples[target_locale]
                )

                for j, record in enumerate(batch_records):
                    record_id = record['id']
                    source_text = record['source']
                    source_locale = record['source_locale']
                    translated_text = translated_texts_batch[j] if j < len(translated_texts_batch) else "ERROR: Translation missing"
                    translated_results_for_file.append({
                        "id": record_id,
                        "source_language": get_language_name(source_locale),
                        "target_language": target_language,
                        "text": source_text,
                        "prediction": translated_text,
                    })

            except Exception as e:
                print(f"\nCRITICAL ERROR: Failed to translate a batch starting with ID '{batch_records[0]['id']}' after multiple retries. Error: {e}")
                for record in batch_records:
                    translated_results_for_file.append({
                        "id": record['id'],
                        "source_language": get_language_name(record['source_locale']),
                        "target_language": get_language_name(record['target_locale']),
                        "text": record['source'],
                        "prediction": "ERROR: Batch translation failed due to API issues/rate limits.",
                    })

            if i + BATCH_SIZE < len(data_to_translate) or overall_pbar.n < len(jsonl_files):
                 time.sleep(DELAY_BETWEEN_REQUESTS_SECONDS)

        with open(outfile_path, 'w', encoding='utf-8') as f:
            for res in translated_results_for_file:
                f.write(json.dumps(res, ensure_ascii=False) + '\n')

        print(f"\nTranslations for {filename} saved to {outfile_path}")

    overall_pbar.close()
    print("All files processed and translations saved (few-shot).")

In [41]:
few_shot_prompt_template = PromptTemplate(
    input_variables=["source_texts_json", "target_language", "few_shot_examples_json"],
    template=FEW_SHOT_PROMPT_TEMPLATE_STRING,
)

few_shot_eval(
    few_shot_prompt_template,
    "few-shot",
    few_shot_examples_dict
)

Processing files (few-shot):   0%|          | 0/5 [00:00<?, ?it/s]

Sending prompt to model: 
Your task is to translate English sentences into Arabic.
Below are a few examples of English sentences with named entities translated correctly into Arabic.
Pay close attention to how named entities (e.g., people, organizations, locations, product names) are translated correctly and consistently.

Examples:
[{"source": "What is the seventh tallest mountain in North America?", "target": "ما سابع أعلى جبل في أمريكا الشمالية؟"}, {"source": "Which actor was the star of Titanic and was born in Los Angeles, California?", "target": "مَنْ الممثل الذي لعب دور البطولة في فيلم \"تيتانيك\" وهو من مواليد لوس أنجلوس بكاليفورنيا؟"}, {"source": "What year was the first book of the A Song of Ice and Fire series published?", "target": "في أي عام تم نشر أول كتاب من سلسلة \"أغنية الجليد والنار\"؟"}, {"source": "Who is the youngest current US governor?", "target": "مَن أصغر حاكم ولاية أمريكي حالٍ؟"}, {"source": "How long did it take to build the Lincoln Memorial?", "target": "كم م

Translating ar_AE.jsonl in batches (few-shot):   7%|▋         | 1/15 [00:19<04:34, 19.63s/it]

Sending prompt to model: 
Your task is to translate English sentences into Arabic.
Below are a few examples of English sentences with named entities translated correctly into Arabic.
Pay close attention to how named entities (e.g., people, organizations, locations, product names) are translated correctly and consistently.

Examples:
[{"source": "What is the seventh tallest mountain in North America?", "target": "ما سابع أعلى جبل في أمريكا الشمالية؟"}, {"source": "Which actor was the star of Titanic and was born in Los Angeles, California?", "target": "مَنْ الممثل الذي لعب دور البطولة في فيلم \"تيتانيك\" وهو من مواليد لوس أنجلوس بكاليفورنيا؟"}, {"source": "What year was the first book of the A Song of Ice and Fire series published?", "target": "في أي عام تم نشر أول كتاب من سلسلة \"أغنية الجليد والنار\"؟"}, {"source": "Who is the youngest current US governor?", "target": "مَن أصغر حاكم ولاية أمريكي حالٍ؟"}, {"source": "How long did it take to build the Lincoln Memorial?", "target": "كم م

Translating ar_AE.jsonl in batches (few-shot):  13%|█▎        | 2/15 [00:37<04:03, 18.73s/it]

...

Sending prompt to model: 
Your task is to translate English sentences into Italian.
Below are a few examples of English sentences with named entities translated correctly into Italian.
Pay close attention to how named entities (e.g., people, organizations, locations, product names) are translated correctly and consistently.

Examples:
[{"source": "What year was the first book of the A Song of Ice and Fire series published?", "target": "In che anno venne pubblicato il primo libro della serie Cronache del Ghiaccio e del Fuoco?"}, {"source": "Which actor was Stephenie Meyers first choice to play Edward Cullen in the movie Twilight?", "target": "Quale attore era stata la prima scelta di Stephanie Meyer per interpretare Edward Cullen nel film Twilight?"}, {"source": "Which river is longer than the Mississippi River?", "target": "Quale fiume è più lungo del Mississippi?"}, {"source": "What is the longest lake in the world?", "target": "Qual è il lago più lungo del mondo?"}, {"source": "How m

Translating it_IT.jsonl in batches (few-shot):  93%|█████████▎| 14/15 [04:18<00:18, 18.37s/it]

Sending prompt to model: 
Your task is to translate English sentences into Italian.
Below are a few examples of English sentences with named entities translated correctly into Italian.
Pay close attention to how named entities (e.g., people, organizations, locations, product names) are translated correctly and consistently.

Examples:
[{"source": "What year was the first book of the A Song of Ice and Fire series published?", "target": "In che anno venne pubblicato il primo libro della serie Cronache del Ghiaccio e del Fuoco?"}, {"source": "Which actor was Stephenie Meyers first choice to play Edward Cullen in the movie Twilight?", "target": "Quale attore era stata la prima scelta di Stephanie Meyer per interpretare Edward Cullen nel film Twilight?"}, {"source": "Which river is longer than the Mississippi River?", "target": "Quale fiume è più lungo del Mississippi?"}, {"source": "What is the longest lake in the world?", "target": "Qual è il lago più lungo del mondo?"}, {"source": "How m

Processing files (few-shot): 100%|██████████| 5/5 [23:48<00:00, 285.62s/it]3<00:00, 17.25s/it]


Translations for it_IT.jsonl saved to data/predictions\gemini-2.0-flash\validation\few-shot\it_IT.jsonl
All files processed and translations saved (few-shot).


In [42]:
calculate_scores("few-shot")

Using default `ModelCheckpoint`. Consider installing `litmodels` package to enable `LitModelCheckpoint` for automatic upload to the Lightning model registry.


GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


All references have a corresponding prediction
Created 1177 instances


Predicting DataLoader 0: 100%|██████████| 37/37 [02:29<00:00,  4.03s/it]
Using default `ModelCheckpoint`. Consider installing `litmodels` package to enable `LitModelCheckpoint` for automatic upload to the Lightning model registry.

Using default `ModelCheckpoint`. Consider installing `litmodels` package to enable `LitModelCheckpoint` for automatic upload to the Lightning model registry.
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


Average COMET score: 90.46
Loaded 722 instances.
Loaded 722 predictions.
All references have a corresponding prediction
Created 1260 instances


...

Average COMET score: 92.35
Loaded 739 instances.
Loaded 739 predictions.
All references have a corresponding prediction
Created 1316 instances


Predicting DataLoader 0: 100%|██████████| 42/42 [03:22<00:00,  4.82s/it]
Using default `ModelCheckpoint`. Consider installing `litmodels` package to enable `LitModelCheckpoint` for automatic upload to the Lightning model registry.

Using default `ModelCheckpoint`. Consider installing `litmodels` package to enable `LitModelCheckpoint` for automatic upload to the Lightning model registry.
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


Average COMET score: 89.91
Loaded 724 instances.
Loaded 724 predictions.
All references have a corresponding prediction
Created 1268 instances


Predicting DataLoader 0: 100%|██████████| 40/40 [02:58<00:00,  4.47s/it]

Average COMET score: 91.63
Loaded 730 instances.
Loaded 730 predictions.


# Chain of Thought Prompting

In [14]:
COT_PROMPT_TEMPLATE_STRING = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """You are an expert linguist and translator. Your primary goal is to translate English sentences into {target_language}.
            You must pay special attention to identifying and accurately translating named entities within the sentence.

            Here's your step-by-step Chain-of-Thought process for each translation.
            Perform these steps internally, and then provide only the final JSON output as requested below.

            1.  **Identify Named Entities:**
                Go through each English sentence and meticulously identify all named entities.
                Named entities include, but are not limited to, persons, organizations, locations, dates, times, and specific titles.

            2.  **Determine Entity Type:**
                For each named entity identified in step 1, classify its specific type.
                Examples of types include: Person, Organization (ORG), Location (LOC), Date, Time, Title, Musical work, Artwork, 
                Food, Animal, Plant, Book, Book series, Fictional entity, Landmark, Movie, Place of worship,
                Natural place, TV series.
                This classification provides crucial context for accurate translation.

            3.  **Contextual Translation of Named Entities:**
                For each identified named entity, determine its most appropriate translation into {target_language}.
                Consider the entity's type and the overall context of the sentence.
                - For proper nouns (like names of people or specific places), often a transliteration or the commonly accepted international form is best.
                - For organizations, use their official translated name if available, otherwise transliterate or provide a descriptive translation.
                - For dates and times, translate them according to the {target_language}'s conventions.

            4.  **Translate Remaining Sentence:**
                Translate all parts of the English sentence that are NOT named entities.
                Ensure that this translation is grammatically correct, natural-sounding, and culturally appropriate for {target_language}.

            5.  **Combine and Finalize Translation:**
                Integrate the accurately translated named entities (from step 3) back into the translated sentence (from step 4).
                Construct the final, complete, and fluent translation of the original English sentence.

            Your final output MUST be a JSON array of objects. Each object MUST contain two fields: "id" (the original sentence ID) and "translation" (the complete translated sentence).
            Do NOT include any other text, reasoning steps, or formatting outside of the JSON array.
            Example for JSON output:
            ```json
            [
              {{"id": "sentence_id_1", "translation": "Translated sentence 1."}},
              {{"id": "sentence_id_2", "translation": "Translated sentence 2."}}
            ]
            ```
            """
        ),
        ("human", "Translate the following English sentences into {target_language}:\n{source_texts_json}"),
    ]
)

In [15]:
@retry(
    wait=wait_random_exponential(multiplier=1, min=DELAY_BETWEEN_REQUESTS_SECONDS, max=60),
    stop=stop_after_attempt(5),
    retry=retry_if_exception_type(Exception)
)
def get_translated_content_with_retries_using_cot(source_records_batch, target_language, llm_instance, template):
    global api_request_count

    input_json_for_prompt = [{"id": rec['id'], "text": rec['source']} for rec in source_records_batch]
    source_texts_json_str = json.dumps(input_json_for_prompt, ensure_ascii=False)

    if isinstance(template, ChatPromptTemplate):
        response = llm_instance.invoke(template.format_messages(
            target_language=target_language,
            source_texts_json=source_texts_json_str
        ))
        raw_content = response.content
    else:
        prompt_to_send = template.format(
            source_texts_json=source_texts_json_str,
            target_language=target_language
        )
        response = llm_instance.invoke(prompt_to_send)
        raw_content = response.content

    api_request_count += 1

    try:
        cleaned_content = raw_content.strip()
        if cleaned_content.startswith("```json") and cleaned_content.endswith("```"):
            cleaned_content = cleaned_content[7:-3].strip()
        
        translated_outputs = json.loads(cleaned_content)

        print(translated_outputs)
        
        if not isinstance(translated_outputs, list) or \
           not all(isinstance(item, dict) and 'id' in item and 'translation' in item for item in translated_outputs):
            raise ValueError("Model did not return a valid JSON array of translation objects.")

        translated_dict = {item['id']: item['translation'] for item in translated_outputs}

        ordered_translations = []
        for record in source_records_batch:
            ordered_translations.append(translated_dict.get(record['id'], "ERROR: ID not found in JSON output"))
        
        return ordered_translations

    except json.JSONDecodeError as e:
        print(f"JSON Decode Error: {e}. Raw response: {raw_content.strip()}")
        # Fallback to a list of error messages for the batch
        return ["ERROR: Invalid JSON response from model"] * len(source_records_batch)
    except ValueError as e:
        print(f"Value Error (JSON format issue): {e}. Raw response: {raw_content.strip()}")
        return ["ERROR: Invalid JSON structure from model"] * len(source_records_batch)
    except Exception as e:
        print(f"An unexpected error occurred while processing model output: {e}. Raw response: {raw_content.strip()}")
        return ["ERROR: Unexpected issue processing model output"] * len(source_records_batch)

def translation_eval_cot(template, template_id, few_shot_examples=None):
    overall_pbar = tqdm.tqdm(jsonl_files, desc=f"Processing files ({template_id})")

    output_prediction_sub_dir = os.path.join(output_prediction_dir, template_id)
    os.makedirs(output_prediction_sub_dir, exist_ok=True)

    for file_path in overall_pbar:
        filename = os.path.basename(file_path)
        outfile_path = os.path.join(output_prediction_sub_dir, filename)

        data_to_translate = []
        with open(file_path, 'r', encoding='utf-8') as f:
            for line in f:
                data_to_translate.append(json.loads(line))

        translated_results_for_file = []

        for i in tqdm.tqdm(range(0, len(data_to_translate), BATCH_SIZE), desc=f"Translating {filename} in batches", leave=False):
            batch_records = data_to_translate[i : i + BATCH_SIZE]

            target_locale = batch_records[0]['target_locale']
            target_language = get_language_name(target_locale)

            if isinstance(template, ChatPromptTemplate):
                current_template = template
            else:
                if few_shot_examples:
                    lang_specific_examples = [
                        ex for ex in few_shot_examples if get_language_name(ex['target_locale']) == target_language
                    ]
                    formatted_examples = json.dumps([
                        {"source": ex['source'], "target": ex['target']} for ex in lang_specific_examples
                    ], ensure_ascii=False)
                    current_template = template.partial(few_shot_examples_json=formatted_examples)
                else:
                    current_template = template


            try:
                translated_texts_batch = get_translated_content_with_retries_using_cot(
                    batch_records, target_language, llm, current_template
                )

                for j, record in enumerate(batch_records):
                    record_id = record['id']
                    source_text = record['source']
                    source_locale = record['source_locale']
                    translated_text = translated_texts_batch[j] if j < len(translated_texts_batch) else "ERROR: Translation missing"
                    translated_results_for_file.append({
                        "id": record_id,
                        "source_language": get_language_name(source_locale),
                        "target_language": target_language,
                        "text": source_text,
                        "prediction": translated_text,
                    })

            except Exception as e:
                print(f"\nCRITICAL ERROR: Failed to translate a batch starting with ID '{batch_records[0]['id']}' after multiple retries. Error: {e}")
                for record in batch_records:
                    translated_results_for_file.append({
                        "id": record['id'],
                        "source_language": get_language_name(record['source_locale']),
                        "target_language": get_language_name(record['target_locale'] ),
                        "text": record['source'],
                        "prediction": "ERROR: Batch translation failed due to API issues/rate limits.",
                    })

            if i + BATCH_SIZE < len(data_to_translate) or overall_pbar.n < len(jsonl_files):
                 time.sleep(DELAY_BETWEEN_REQUESTS_SECONDS)

        with open(outfile_path, 'w', encoding='utf-8') as f:
            for res in translated_results_for_file:
                f.write(json.dumps(res, ensure_ascii=False) + '\n')

        print(f"\nTranslations for {filename} saved to {outfile_path}")

    overall_pbar.close()
    print(f"All files processed for {template_id} and translations saved.")

In [16]:
translation_eval_cot(COT_PROMPT_TEMPLATE_STRING, "cot")
calculate_scores("cot")

[{'id': 'Q101438737_0', 'translation': 'ما هو النوع الذي يندرج تحته كتاب أرض الميعاد؟'}, {'id': 'Q101438737_1', 'translation': 'من هو مؤلف العمل الفني أرض الميعاد؟'}, {'id': 'Q101438737_2', 'translation': 'متى تم نشر كتاب أرض الميعاد؟'}, {'id': 'Q10298666_0', 'translation': 'ما هو نوع فيلم كيف تروض تنينك 2؟'}, {'id': 'Q10298666_1', 'translation': 'في أي عام تم إصدار فيلم كيف تروض تنينك 2؟'}, {'id': 'Q10298666_2', 'translation': 'هل فيلم كيف تروض تنينك 2 فيلم رسوم متحركة؟'}, {'id': 'Q1031002_0', 'translation': 'من هو فنان القصص المصورة البلجيكي الشهير الذي ابتكر مغامرات تان تان؟'}, {'id': 'Q1031002_1', 'translation': 'ما هو نوع مغامرات تان تان؟'}, {'id': 'Q1031002_2', 'translation': 'كم عدد المواسم الموجودة في مغامرات تان تان؟'}, {'id': 'Q1044613_0', 'translation': 'ما هي الوظيفة الرئيسية لمحطة هلسنكي المركزية للسكك الحديدية؟'}, {'id': 'Q1044613_1', 'translation': 'كيف تصف الهندسة المعمارية لمحطة هلسنكي المركزية للسكك الحديدية؟'}, {'id': 'Q1044613_2', 'translation': 'كم عدد الأرصفة المو

[{'id': 'Q1137661_0', 'translation': 'هل يمكنك تقديم وصف موجز لدير غراتشانيكا؟'}, {'id': 'Q1137661_1', 'translation': 'ما هو نوع المكان الذي يمثله دير غراتشانيكا؟'}, {'id': 'Q1137661_2', 'translation': 'ما هي أهمية دير غراتشانيكا؟'}, {'id': 'Q11446789_0', 'translation': 'هل تدور أحداث Elegant Yokai Apartment Life في إطار حديث أم تاريخي؟'}, {'id': 'Q11446789_1', 'translation': 'من هو مؤلف Elegant Yokai Apartment Life؟'}, {'id': 'Q11446789_2', 'translation': 'كم عدد الكتب الموجودة في سلسلة Elegant Yokai Apartment Life؟'}, {'id': 'Q1152214_0', 'translation': 'ما هي أهمية عنوان The Whisperer in Darkness بالنسبة للرواية؟'}, {'id': 'Q1152214_1', 'translation': 'من هو مؤلف رواية The Whisperer in Darkness؟'}, {'id': 'Q1152214_2', 'translation': 'كيف يمكنك تلخيص حبكة رواية The Whisperer in Darkness؟'}, {'id': 'Q1167737_0', 'translation': 'ما هو التركيز الرئيسي لكتاب مستقبل وهم لسيغموند فرويد؟'}, {'id': 'Q1167737_1', 'translation': 'كيف تصف موضوع عمل سيغموند فرويد الفني، مستقبل وهم؟'}, {'id': 'Q

[{'id': 'Q1248011_2', 'translation': 'كم عدد مواسم المسلسل التلفزيوني أبنائي الثلاثة التي تم إنتاجها؟'}, {'id': 'Q1254986_0', 'translation': 'من هم الشخصيات الرئيسية في دراغون بول زد: شجرة القوة؟'}, {'id': 'Q1254986_1', 'translation': 'متى تم إصدار دراغون بول زد: شجرة القوة؟'}, {'id': 'Q1254986_2', 'translation': 'ما هو نوع دراغون بول زد: شجرة القوة؟'}, {'id': 'Q13099103_0', 'translation': 'كيف تقارن طعم حليب القنب مع أنواع الحليب النباتية الأخرى؟'}, {'id': 'Q13099103_1', 'translation': 'ما هي المكونات الرئيسية في حليب القنب؟'}, {'id': 'Q13099103_2', 'translation': 'هل حليب القنب مناسب للأشخاص الذين لديهم قيود غذائية؟'}, {'id': 'Q131074_0', 'translation': 'متى تم إصدار فيلم سيد الخواتم: عودة الملك؟'}, {'id': 'Q131074_1', 'translation': 'من أخرج فيلم 2003، سيد الخواتم: عودة الملك؟'}, {'id': 'Q131074_2', 'translation': 'إلى أي نوع ينتمي فيلم سيد الخواتم: عودة الملك؟'}, {'id': 'Q1316142_0', 'translation': 'هل يمكنك تقديم وصف موجز لشخصية ألفي أتكنز؟'}, {'id': 'Q1316142_1', 'translation': '

[{'id': 'Q1545219_0', 'translation': 'هل الأب براون شخصية حقيقية أم خيالية؟'}, {'id': 'Q1545219_1', 'translation': 'ما هي مهنة الأب براون؟'}, {'id': 'Q1545219_2', 'translation': 'في أي بلد تم إنشاء الأب براون؟'}, {'id': 'Q1548310_0', 'translation': 'هل يمكن زيارة مسجد بروكسل الكبير من قبل الجمهور؟'}, {'id': 'Q1548310_1', 'translation': 'أين يقع مسجد بروكسل الكبير؟'}, {'id': 'Q1548310_2', 'translation': 'ما هو الغرض من مسجد بروكسل الكبير؟'}, {'id': 'Q15547365_0', 'translation': 'كيف يعكس مسجد ناجياينغ العمارة الصينية؟'}, {'id': 'Q15547365_1', 'translation': 'ما هو الغرض الأساسي من مسجد ناجياينغ؟'}, {'id': 'Q15547365_2', 'translation': 'متى تم بناء مسجد ناجياينغ؟'}, {'id': 'Q15616479_0', 'translation': 'من هم الشخصيات الرئيسية في بطاريق مدغشقر؟'}, {'id': 'Q15616479_1', 'translation': 'متى تم إصدار فيلم بطاريق مدغشقر؟'}, {'id': 'Q15616479_2', 'translation': 'ما هو نوع فيلم بطاريق مدغشقر؟'}, {'id': 'Q1572056_0', 'translation': 'ما هي الرياضات التي يتم تمثيلها عادةً في ترشيحات جائزة شخصية ا

[{'id': 'Q1759977_2', 'translation': 'متى نُشرت رواية الفضاء الوحي (Revelation Space) لأول مرة؟'}, {'id': 'Q176061_0', 'translation': 'بأية طرق تساهم لغة تصميم الواقع الافتراضي (Virtual Reality Modeling Language) في تطوير تطبيقات الواقع الافتراضي؟'}, {'id': 'Q176061_1', 'translation': 'كيف تعزز لغة تصميم الواقع الافتراضي (Virtual Reality Modeling Language) تجربة المستخدم في البيئات الافتراضية التفاعلية؟'}, {'id': 'Q176061_2', 'translation': 'ما هي الميزات الرئيسية للغة تصميم الواقع الافتراضي (Virtual Reality Modeling Language) التي تجعلها تنسيق ملفات شائعًا للرسومات ثلاثية الأبعاد؟'}, {'id': 'Q179468_0', 'translation': 'من أين كان شلمنصر الرابع (Shalmaneser IV)؟'}, {'id': 'Q179468_1', 'translation': 'بماذا اشتهر شلمنصر الرابع (Shalmaneser IV)؟'}, {'id': 'Q179468_2', 'translation': 'متى حكم شلمنصر الرابع (Shalmaneser IV)؟'}, {'id': 'Q1810196_0', 'translation': 'ما هو نوع رواية نبيذ الهندباء (Dandelion Wine)؟'}, {'id': 'Q1810196_1', 'translation': 'من كتب كتاب نبيذ الهندباء (Dandelion Wi

...

Average COMET score: 92.21
Loaded 739 instances.
Loaded 739 predictions.
All references have a corresponding prediction
Created 1316 instances


Predicting DataLoader 0: 100%|██████████| 42/42 [04:06<00:00,  5.87s/it]
Using default `ModelCheckpoint`. Consider installing `litmodels` package to enable `LitModelCheckpoint` for automatic upload to the Lightning model registry.
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


Average COMET score: 90.09
Loaded 724 instances.
Loaded 724 predictions.
All references have a corresponding prediction
Created 1268 instances


Predicting DataLoader 0: 100%|██████████| 40/40 [02:58<00:00,  4.47s/it]

Average COMET score: 91.01
Loaded 730 instances.
Loaded 730 predictions.
